In [1]:
import numpy as np
import pandas as pd

# Data preprocessing

### Read data and save to vector

In [2]:
from sklearn.preprocessing import StandardScaler

X_train_df = pd.read_csv('./data/X_train.csv', skiprows=1, header=None)
y_train_df = pd.read_csv('./data/y_train.csv', skiprows=1, header=None)
X_test_df = pd.read_csv('./data/X_test.csv', skiprows=1, header=None)

X_train_full = X_train_df.values[:, 1:]
y_train_full = y_train_df.values[:, 1:]
X_test = X_test_df.values[:, 1:]

print(X_train_full.shape, y_train_full.shape, X_test.shape)

(1212, 832) (1212, 1) (776, 832)


### Train and validation set
Used a 70-30 split, can be changed at `testSize=0.3`

In [3]:
from sklearn.model_selection import train_test_split

testSize = 0.3
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size = testSize, random_state=42)

### Imputing NaN

In [4]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer

mean_imputer = SimpleImputer(strategy='mean')
X_train_imputed = mean_imputer.fit_transform(X_train_full)

### Feature selection

In [5]:
from sklearn.feature_selection import SelectKBest, f_regression

selector = SelectKBest(score_func=f_regression, k='all')
selector.fit(X_train_imputed, y_train_full.ravel())

p_values = selector.pvalues_
scores = selector.scores_

significant_feature_indices = np.where(p_values < 0.05)[0]

print(f"Number of significant features: {len(significant_feature_indices)}")

Number of significant features: 245


/home/diego/src/AML_projects/project1/.venv/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:379: RuntimeWarning: invalid value encountered in sqrt
  X_norms = np.sqrt(row_norms(X.T, squared=True) - n_samples * X_means**2)


In [6]:
# Extract selected features from the original data
X_train_selected = X_train_full[:, significant_feature_indices]
X_val_selected = X_val[:, significant_feature_indices]
X_test_selected = X_test[:, significant_feature_indices]

### Final NaN imputation

In [7]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Impute missing values in the selected features
iterative_imputer = IterativeImputer()
X_train_selected_imputed = iterative_imputer.fit_transform(X_train_selected)
X_val_selected_imputed = iterative_imputer.transform(X_val_selected)
X_test_selected_imputed = iterative_imputer.transform(X_test_selected)

### Scale features

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected_imputed)
X_val_scaled = scaler.transform(X_val_selected_imputed)
X_test_scaled = scaler.transform(X_test_selected_imputed)

In [12]:
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error

lasso = LassoCV(cv=5, random_state=42)
lasso.fit(X_train_scaled, y_train_full.ravel())

y_val_pred = lasso.predict(X_val_scaled)
y_train_pred = lasso.predict(X_train_scaled)
mse = mean_squared_error(y_val, y_val_pred)
print(f"Validation Mean Squared Error: {mse}")

/home/diego/src/AML_projects/project1/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15.896877955274249, tolerance: 9.32950608875129
  model = cd_fast.enet_coordinate_descent_gram(
/home/diego/src/AML_projects/project1/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 11.540313514196896, tolerance: 9.32950608875129
  model = cd_fast.enet_coordinate_descent_gram(
/home/diego/src/AML_projects/project1/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.222512078747968, tolerance: 9.32950608875129
  model = cd_fast.enet_coordinate_descent_gram(
/hom

Validation Mean Squared Error: 70.82209545791454
